In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import psycopg2
import os
import json
import re

In [2]:
def create_connection():
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    cursor = conn.cursor()

    return conn, cursor

In [3]:
def create_fb_data_table():
    #create connection to db
    conn, cursor = create_connection()

    table_check_sql = """
    SELECT EXISTS (
        SELECT FROM pg_tables 
        WHERE schemaname = 'public' 
        AND tablename = 'fb_data'
    );
    """

    create_table_sql = """
            CREATE TABLE fb_data (
                id SERIAL PRIMARY KEY,
                text TEXT NOT NULL,
                source TEXT NOT NULL,
                UNIQUE (text)
            );
            """
    try:
        cursor.execute(table_check_sql)
        table_exists = cursor.fetchone()[0]
        if not table_exists:
            cursor.execute(create_table_sql)
            conn.commit()
            print("Table created successfully")
        else:
            print("Table exists")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    cursor.close()
    conn.close()

In [4]:
def get_facebook_posts(username, post_count_break):
    options = Options()
    options.headless = True
    driver = webdriver.ChromiumEdge(options=options)

    driver.get(f"https://www.facebook.com/{username}/posts")

    keywords = {
        "election", "vote", "voting", "candidate", "campaign",
        "politician", "politics", "government", "congress", "senate", 
        "house of representatives", "white house", "president", 
        "vice president", "democrat", "republican", "liberal", "conservative",
        "policy", "legislation", "bill", "law", "executive order", "constitution",
        "supreme court", "judiciary", "governor", "mayor", "local government",
        "senator", "representative", "parliament", "prime minister", "ambassador",
        "diplomacy", "foreign policy", "domestic policy", "budget", "taxes", "taxation",
        "healthcare", "immigration", "border", "national security", "military",
        "war", "conflict", "peace", "international relations", "trade", "sanctions",
        "lobbying", "political party", "activism", "protest", "movement", "rights",
        "freedom", "civil rights", "human rights", "reform", 
        "corruption", "scandal", "ethics", "voter suppression", "gerrymandering",
        "climate change", "environmental policy", "energy policy", "education policy",
        "economic policy", "labor", "wages", "social security", "medicare", "medicaid",
        "welfare", "public assistance", "subsidy", "entitlement", "budget deficit",
        "debt ceiling", "public debt", "federal reserve", "central bank", "inflation",
        "unemployment", "job creation", "economic growth", "GDP", "trade deficit",
        "export", "import", "tariff", "free trade", "fair trade", "monetary policy",
        "fiscal policy", "regulation", "deregulation", "privatization", "nationalization",
        "kamala", "trump", "vance", "walz", "democrats", "republicans"
    }

    insert_sql = """
    INSERT INTO fb_data (
        text, source
    ) VALUES (
        %s, %s
    )
    """

    seen_posts = set()
    conn,cursor = create_connection()
    conn.autocommit = True
    post_count = 0

    while post_count < post_count_break:

        for _ in range(10):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.5)

        elements = driver.find_elements(By.XPATH, '//div[@data-ad-preview="message"]')
        for element in elements:
            content = element.text
            if content not in seen_posts:
                seen_posts.add(content)
                found_keywords = []
                for keyword in keywords:
                    pattern = r'\b' + re.escape(keyword) + r'\b'
                    if re.search(pattern, content, re.IGNORECASE):
                        found_keywords.append(keyword)

                if found_keywords:
                    try:
                        cursor.execute(insert_sql,(content, username))
                        post_count+=1
                    except(Exception, psycopg2.DatabaseError) as error:
                        print(error)


    driver.quit()
    conn.close()
    cursor.close()    

In [5]:
left_bias = ["nytimes","cnn","nbc"] #done nytimes, cnn
right_bias = ["FoxNews","DailyMail","NYPost"] #done FoxNews
center_bias = ["bbcnews","Reuters","APNews"] #done bbcnews
username = "DailyMail"  
post_count_break = 1000

create_fb_data_table()
get_facebook_posts(username, post_count_break)


Table exists


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.86)
Stacktrace:
	GetHandleVerifier [0x00007FF6E965B632+29090]
	(No symbol) [0x00007FF6E95CE6E9]
	(No symbol) [0x00007FF6E948B1CA]
	(No symbol) [0x00007FF6E945FAF5]
	(No symbol) [0x00007FF6E950E2E7]
	(No symbol) [0x00007FF6E9525EE1]
	(No symbol) [0x00007FF6E9506493]
	(No symbol) [0x00007FF6E94D09B1]
	(No symbol) [0x00007FF6E94D1B11]
	GetHandleVerifier [0x00007FF6E997883D+3294125]
	GetHandleVerifier [0x00007FF6E99C4423+3604371]
	GetHandleVerifier [0x00007FF6E99BA2E7+3563095]
	GetHandleVerifier [0x00007FF6E9716F16+797318]
	(No symbol) [0x00007FF6E95D986F]
	(No symbol) [0x00007FF6E95D5454]
	(No symbol) [0x00007FF6E95D55E0]
	(No symbol) [0x00007FF6E95C4A7F]
	BaseThreadInitThunk [0x00007FFC184D257D+29]
	RtlUserThreadStart [0x00007FFC19CAAF28+40]
